# Sports Dataset

10,000 random photos from soccer matches captured at a single timestamp make up the dataset. Every snapshot has a number of elements that offer details on the current match. The objective of the dataset is to capture the evolving dynamics of a soccer game, and to make predictions for the number of goals that will be scored for the remaining playing time.

# Dataset Structure

The dataset is presented in a single file with 10,000 rows and 20 columns, each row representing a snapshot of a soccer match, and each column representing a specific attribute or feature of that snapshot.

Number of Instances (Rows): 10,000 instances 
Number of Features (Columns): 20 features/columns 

# Feature Representation

The features include identifiers such as uuid, and dynamic match-related details such as the current minute of play, home and away team scores, yellow and red cards for both teams, the number of attacks and dangerous attacks by each team, corners awarded, shots on and off target for both teams, and the ball possession percentage for both the home and away teams until the current minute.

# Libraries

- pandas
- numpy
- matplotlib
- pickle
- os
- sklearn.neighbors KNeighborsClassifier
- sklearn.metrics accuracy_score, classification_report, mean_squared_error, r2_score
- sklearn.model_selection cross_val_predict, cross_val_score, train_test_split
- sklearn preprocessing 
- sklearn.linear_model LinearRegression

In [84]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import pickle
import os
import seaborn as sns
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, classification_report, mean_squared_error, r2_score, confusion_matrix
from sklearn.model_selection import cross_val_predict, cross_val_score
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Lasso
from sklearn.linear_model import LassoCV

In [85]:
df = pd.read_csv(filepath_or_buffer = 'Dataset 3 - Sports Dataset/sports.csv', header = 0)
df

,uuid,current_minute,home_score,away_score,home_yellow_cards,away_yellow_cards,home_red_cards,away_red_cards,home_attacks,away_attacks,...,away_dangerous_attacks,home_corners,away_corners,home_off_target,away_off_target,home_on_target,away_on_target,home_possession,away_possession,final_delta
0,2f9dd904-8d9d-4aa0-a9d9-4e0f3e50d742,94,1,2,0,0,0,0,39,45,...,26,4,1,4,0,4,6,48,52,1
1,869c1420-9153-4d73-8d79-5db31a8a80a4,91,2,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,50,50,1
2,549e25a3-4c93-459d-b115-fb303273c78a,3,0,0,0,0,0,0,3,1,...,0,0,0,0,0,0,0,62,38,4
3,287b4be5-9163-4e2e-895d-d11052a97410,17,0,0,1,0,0,0,41,46,...,25,1,0,0,2,0,0,55,45,2
4,a607da6a-d6b2-4652-b7a5-6c8e4b61c2b8,76,1,0,1,0,0,0,69,78,...,46,1,1,0,3,4,4,45,55,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6995,838f35c5-afa3-474b-876e-063fd49d4326,32,0,0,0,0,0,0,16,25,...,13,0,1,1,2,0,1,45,55,0
6996,4bcfd248-5f39-4af2-b465-5358c9e7088e,101,3,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,50,50,0
6997,deccdc2c-625e-495a-9b06-02d540d4cd24,78,1,0,0,0,0,0,71,60,...,40,7,0,5,5,7,0,61,39,2
6998,94a20308-55d8-47cb-8f8b-d2702a531c74,72,2,0,0,0,0,0,85,66,...,20,3,0,4,1,2,1,52,48,0


# Data Preprocessing
Our objective for this project is to use the data given in the dataset to predict whether 1 or more goals can be scored in the remaining playing time of the match.

In [86]:
# Handling Missing Data
df.dropna(inplace=True)
try:
    df = df.drop(['uuid'], axis=1)
except:
    print('uuid dropped already')
# Feature Scaling
numerical_features = ['current_minute', 'home_score', 'away_score', 'home_yellow_cards', 'away_yellow_cards',
                      'home_red_cards', 'away_red_cards', 'home_attacks', 'away_attacks', 'home_dangerous_attacks',
                      'away_dangerous_attacks', 'home_corners', 'away_corners', 'home_off_target', 'away_off_target',
                      'home_on_target', 'away_on_target', 'home_possession', 'away_possession']

In [87]:
df = df.query("0<=current_minute<=105")
# dropping any data out of the current game

In [88]:
df = df.query("44>= current_minute | current_minute >= 60")

# remove rows during halftime. probably an exhibition match.
# actual game time is 90 minutes. last minute of first half should be minute 44.half time lasts 15.
# half time happens around minute 45
fun = lambda t: t-15 if t >=60 else t
df['current_minute'].transform(fun)
#

0       79
1       76
2        3
3       17
4       61
        ..
6995    32
6996    86
6997    63
6998    57
6999     8
Name: current_minute, Length: 6577, dtype: int64

In [73]:
# Create variables for the remaining time and the target
# the target variable indicates whether 1 goal or more will be scored with the remaining time of the match. it shows binary values
df['remaining_time'] = df['current_minute'].max() - df['current_minute']
df['target'] = (df['home_score'] + df['away_score']).apply(lambda x: 1 if x > 0 else 0)

In [77]:
df1 = df.query("44< current_minute< 60")

In [ ]:
X = df.drop(['target'], axis=1)
y = df['target']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [69]:
df1 = df.query("48<current_minute <=50")
df1.sort_values(by=['current_minute'])

,current_minute,home_score,away_score,home_yellow_cards,away_yellow_cards,home_red_cards,away_red_cards,home_attacks,away_attacks,home_dangerous_attacks,...,away_corners,home_off_target,away_off_target,home_on_target,away_on_target,home_possession,away_possession,final_delta,remaining_time,target
28,49,1,0,4,4,0,0,48,29,35,...,1,9,1,2,1,66,34,5,56,1
6674,49,1,1,0,0,0,1,37,29,39,...,1,2,3,2,1,61,39,1,56,1
6249,49,2,1,0,2,0,0,32,40,30,...,1,0,0,2,1,46,54,3,56,1
5891,49,0,0,0,0,0,0,57,64,47,...,2,4,3,2,1,55,45,1,56,0
5691,49,1,0,0,0,0,0,35,18,28,...,4,6,2,2,3,63,37,3,56,1
5514,49,0,2,0,0,0,0,21,29,11,...,0,0,1,0,4,52,48,1,56,1
4860,49,3,0,1,0,0,0,35,38,22,...,1,7,4,6,3,58,42,3,56,1
4774,49,0,2,0,0,0,0,0,0,0,...,0,0,0,0,0,50,50,0,56,1
4437,49,2,0,0,0,0,0,37,39,18,...,1,2,4,3,0,46,54,2,56,1
3534,49,0,0,0,0,0,0,72,31,53,...,1,3,1,1,0,68,32,0,56,0


In [ ]:
# Current shape of our X and y
print("Shape of X:", X.shape)
print("Shape of y:", y.shape)

# Exploratory Data Analysis

This is the content of the dataset

In [ ]:
print("First 5 rows of the dataset:")
df.head()

In [ ]:
df.plot(x='remaining_time', y='target', style='o')
plt.xlabel('remaining_time')
plt.ylabel('target')
plt.show()

Basic summary statistics for numerical values

In [ ]:
print("\nSummary statistics of numerical features:")
df.describe()

Scatterplot matrix for pairs of numerical features

In [ ]:
print("\nPairplot for numerical features:")
scatter_matrix = pd.plotting.scatter_matrix(df[['home_score', 'away_score', 'home_yellow_cards', 'away_yellow_cards', 'home_possession', 'away_possession']], figsize=(12, 8))
plt.show()

Histogram that shows the distribution of home and away scores

In [ ]:
print("\nDistribution of home and away scores:")
plt.figure(figsize=(12, 6))
plt.hist(df['home_score'], bins=20, alpha=0.5, label='Home Score', color='blue')
plt.hist(df['away_score'], bins=20, alpha=0.5, label='Away Score', color='red')
plt.title('Distribution of Home and Away Scores')
plt.legend()
plt.show()

# Linear Regression Model

In [ ]:
# Initialize Linear Regression Model
linreg = LinearRegression()
linreg.fit(X_train, y_train)

First, we will perform Linear Regression on the Training set.

In [ ]:
# Make predictions on the training set
train_predictions = linreg.predict(X_train)

# Convert predictions to binary (0 or 1)
train_predictions_binary = (train_predictions >= 0.5).astype(int)

To evaluate the model, we will get the training accuracy

In [ ]:
# Evaluate the model on the training set
train_accuracy = accuracy_score(y_train, train_predictions_binary)
train_conf_matrix = confusion_matrix(y_train, train_predictions_binary)
train_classification_rep = classification_report(y_train, train_predictions_binary)

print("Train Accuracy:", train_accuracy)
print("Train Confusion Matrix:\n", train_conf_matrix)
print("Train Classification Report:\n", train_classification_rep)

Based on the results, we can see that we got an accuracy of about 93% which indicates that the model predicted the target correctly for 93% of the instances in the train set. 

Train Confusion Matrix:

True Positives: 3331
True Negatives: 1877
False Positives: 53
False Negatives: 339

In terms of accuracy, the model has a good performance on the train set in terms of precision, recall, and F1-score for both classes. Class 0 has slightly higher precision, while class 1 has slightly higher recall. The weighted average F1-score is 0.93, indicating good overall performance.

In [ ]:
# Calculate RMSE and R2 Score for the training set
train_rmse = np.sqrt(mean_squared_error(y_train, train_predictions))
train_r2 = r2_score(y_train, train_predictions)

print(f"Training RMSE: {train_rmse}")
print(f"Training R2 Score: {train_r2}")

The rmse is at about 33% and the R2 score is about 50%. The R2 score indicates that the model explains about 50% of the variance in the target in the train set. The model needs improvement, especially in terms of reducing the RMSE.

Now that we have the train accuracy, rmse, and r-squared values, let's visualize our results on a graph.

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(12, 4))

# Confusion Matrix
sns.heatmap(train_conf_matrix, annot=True, fmt='d', cmap='Blues', cbar=False, ax=axes[0])
axes[0].set_xlabel('Predicted')
axes[0].set_ylabel('True')
axes[0].set_title('Confusion Matrix - Training Set')

# Scatter Plot for Actual vs. Predicted Values
axes[1].scatter(y_train, train_predictions, alpha=0.5)
axes[1].set_xlabel('Actual Values')
axes[1].set_ylabel('Predicted Values')
axes[1].set_title('Actual vs. Predicted Values - Training Set')
plt.tight_layout()
plt.show()

We will now create a validation set to further assess our model.

In [ ]:
# Split the data again for the validation set
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
val_predictions = linreg.predict(X_val)
val_predictions_binary = (val_predictions >= 0.5).astype(int)

We have created the validation set. Now we will evaluate the model on it.

In [ ]:
# Evaluate the model on the validation set
val_accuracy = accuracy_score(y_val, val_predictions_binary)
val_conf_matrix = confusion_matrix(y_val, val_predictions_binary)
val_classification_rep = classification_report(y_val, val_predictions_binary)

print("\nValidation Accuracy:", val_accuracy)
print("Validation Confusion Matrix:\n", val_conf_matrix)
print("Validation Classification Report:\n", val_classification_rep)

The model achieved a high accuracy of approximately 91.6% on the validation set. The model performs well on the validation set with high precision, recall, and F1-Score for both classes.

Confusion Matrix:

True Positives: 834
True Negatives: 448
False Positives: 14
False Negatives: 104

The high accuracy, combined with balanced precision and recall, suggests that the model generalizes well to new data. The validation results indicate that the model is performing effectively on unseen data, and its predictions are reliable. 

In [ ]:
# Calculate RMSE and R2 Score for the validation set
val_rmse = np.sqrt(mean_squared_error(y_val, val_predictions))
val_r2 = r2_score(y_val, val_predictions)

print(f"Validation RMSE: {val_rmse}")
print(f"Validation R2 Score: {val_r2}")

In [ ]:
# Confusion Matrix Plot for Validation Set
fig, axes = plt.subplots(1, 2, figsize=(12, 4))

# Confusion Matrix
sns.heatmap(val_conf_matrix, annot=True, fmt='d', cmap='Blues', cbar=False, ax=axes[0])
axes[0].set_xlabel('Predicted')
axes[0].set_ylabel('True')
axes[0].set_title('Confusion Matrix - Validation Set')

# Scatter Plot for Actual vs. Predicted Values - Validation Set
axes[1].scatter(y_val, val_predictions, alpha=0.5)
axes[1].set_xlabel('Actual Values')
axes[1].set_ylabel('Predicted Values')
axes[1].set_title('Actual vs. Predicted Values - Validation Set')

plt.tight_layout()
plt.show()

After evaluating the model on the validation set, we will now evaluate it on the test set.

In [ ]:
# Make predictions on the test set
test_predictions = linreg.predict(X_test)
test_predictions_binary = (test_predictions >= 0.5).astype(int)

In [ ]:
# Evaluate the model on the test set
test_accuracy = accuracy_score(y_test, test_predictions_binary)
test_conf_matrix = confusion_matrix(y_test, test_predictions_binary)
test_classification_rep = classification_report(y_test, test_predictions_binary)

print("Test Accuracy:", test_accuracy)
print("Test Confusion Matrix:\n", test_conf_matrix)
print("Test Classification Report:\n", test_classification_rep)

In [ ]:
# Calculate RMSE and R2 Score for the test set
test_rmse = np.sqrt(mean_squared_error(y_test, test_predictions))
test_r2 = r2_score(y_test, test_predictions)

print(f"Test RMSE: {test_rmse}")
print(f"Test R2 Score: {test_r2}")

In [ ]:
# Confusion Matrix Plot for Test Set
fig, axes = plt.subplots(1, 2, figsize=(12, 4))

# Confusion Matrix
sns.heatmap(test_conf_matrix, annot=True, fmt='d', cmap='Blues', cbar=False, ax=axes[0])
axes[0].set_xlabel('Predicted')
axes[0].set_ylabel('True')
axes[0].set_title('Confusion Matrix - Test Set')

# Scatter Plot for Actual vs. Predicted Values - Test Set
axes[1].scatter(y_test, test_predictions, alpha=0.5)
axes[1].set_xlabel('Actual Values')
axes[1].set_ylabel('Predicted Values')
axes[1].set_title('Actual vs. Predicted Values - Test Set')

plt.tight_layout()
plt.show()

Based on the results of the test set evaluation, we can see that it is consistent with the validation set as both show the same values in terms of accuracy, classification, and confusion matrix. Given the consistency of both results, we can see that we have a robust model. 

Though the model is performing well in terms of accuracy, it still has a shortcoming in capturing the class (1). Further improvements can still be implemented through hyperparameter tuning.

# Hyperparameter Tuning

We Will be using Lasso Regularization on the model to find the best value of alpha, and to try to enhance the model's performance on the test set. 

In [ ]:
alphas = [0.01, 0.1, 1, 10, 100]

# Lasso regression model with cross-validated alpha selection
lasso_cv = LassoCV(alphas=alphas, cv=5)
lasso_cv.fit(X_train, y_train)

# Get the best alpha
best_alpha = lasso_cv.alpha_

# Train the model with the best alpha
lasso_reg_best = Lasso(alpha=best_alpha, random_state=42)
lasso_reg_best.fit(X_train, y_train)

# Make predictions on the test set
test_predictions = lasso_reg_best.predict(X_test)
test_predictions_binary = (test_predictions >= 0.5).astype(int)

# Evaluate the model on the test set
test_accuracy = accuracy_score(y_test, test_predictions_binary)
test_conf_matrix = confusion_matrix(y_test, test_predictions_binary)
test_classification_rep = classification_report(y_test, test_predictions_binary)

print("Best Alpha:", best_alpha)
print("Test Accuracy:", test_accuracy)
print("Test Confusion Matrix:\n", test_conf_matrix)
print("Test Classification Report:\n", test_classification_rep)